In [4]:
!pip install noisereduce
!pip install scipy
!pip install PIL

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [1]:
# Global imports
import os
import glob
from scipy.io import wavfile
from scipy import signal
import re
import noisereduce as nr

c:\Users\herna\anaconda3\envs\Noisereduce\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Path to the directory containing folders with .wav files
root_path = "d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data"

# Define the root path for saving progress
PROGRESS_PATH = os.path.join(os.getcwd(), 'progress.txt')

In [ ]:

# Function to generate spectrogram
def gggenerate_spectrogram(data, rate, save_path, resized_spectrogram_save_path):
    plt.figure(figsize=(10, 4))
    f, t, Sxx = signal.spectrogram(data, fs=rate, nperseg=256, noverlap=128, window='hamming', scaling='density')
    plt.pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), cmap='inferno')  # Adding small value to avoid log(0)
    plt.ylim(0, 8000)  # Adjust frequency range if needed
    plt.axis('off')  # Turn off axis labels
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0, facecolor='black')  # Dark background
    plt.close()

def generate_spectrogram(data, rate, save_path):
    plt.figure(figsize=(10, 4))
    f, t, Sxx = signal.spectrogram(data, fs=rate, nperseg=256, noverlap=128, window='hamming', scaling='spectrum')
    plt.pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), cmap='inferno')  # Adding small value to avoid log(0)
    plt.ylim(0, 8000)  # Adjust frequency range if needed
    plt.axis('off')  # Turn off axis labels
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)  # Save the figure
    plt.close()

def ggenerate_spectrogram(data, rate, save_path, resized_spectrogram_save_path):
    plt.figure(figsize=(10, 4))
    chunk_size = 256
    overlap = 128
    print(data.shape)
    
    num_chunks = (len(data) - overlap) // (chunk_size - overlap)
    max_freq_bins = 0
    
    # Generate spectrograms for each chunk and determine the maximum number of frequency bins
    for i in range(0, len(data) - chunk_size + overlap, chunk_size - overlap):
        chunk = data[i:i + chunk_size]
        f, t, Sxx = signal.spectrogram(chunk, fs=rate, nperseg=64, noverlap=0, window='hamming', scaling='spectrum', nfft=64)
        max_freq_bins = max(max_freq_bins, Sxx.shape[0])
    
    # Pre-allocate array to hold the merged spectrogram
    merged_spectrogram = np.zeros((max_freq_bins, num_chunks * (chunk_size // 2 + 1)))

    # Generate spectrograms for each chunk and concatenate them
    col_idx = 0
    for i in range(0, len(data) - chunk_size + overlap, chunk_size - overlap):
        chunk = data[i:i + chunk_size]
        f, t, Sxx = signal.spectrogram(chunk, fs=rate, nperseg=64, noverlap=0, window='hamming', scaling='spectrum', nfft=64)
        merged_spectrogram[:Sxx.shape[0], col_idx:col_idx + Sxx.shape[1]] = Sxx
        col_idx += Sxx.shape[1]

    f, t, _ = signal.spectrogram(data[:chunk_size], fs=rate, nperseg=64, noverlap=0, window='hamming', scaling='spectrum', nfft=64)
    plt.pcolormesh(t, f, 10 * np.log10(merged_spectrogram + 1e-10), cmap='inferno')  # Adding small value to avoid log(0)
    plt.ylim(0, 8000)  # Adjust frequency range if needed
    plt.axis('off')  # Turn off axis labels
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0, facecolor='black')  # Dark background
    plt.close()
# Function to resize image
def resize_image(image_path, target_size, save_path):
    img = Image.open(image_path)
    img_resized = img.resize(target_size, resample=Image.Resampling.NEAREST)
    img_resized.save(save_path)

In [3]:
# Function to save progress
def save_progress( wav_file_path):
    with open(PROGRESS_PATH, 'w') as file:
        file.write(wav_file_path)

# Function to load progress
def load_progress():
    progress_file = PROGRESS_PATH
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as file:
            return file.read().strip()
    else:
        return None

In [4]:
def extract_numbers_from_string(input_string):
    # Regular expression to match numbers
    pattern = r'\d+'
    # Find all numbers in the input string
    numbers = re.findall(pattern, input_string)
    return numbers

In [7]:
# Generate preprocessed files
# Load progress
last_wav_file_path = load_progress()
def processAudio():
  for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)
    if os.path.isdir(folder_path):
      if last_wav_file_path!= None and extract_numbers_from_string(folder_path) < extract_numbers_from_string(last_wav_file_path):
        continue
      for wav_file_path in glob.glob(os.path.join(folder_path, '*.wav')):
        # Save progress
        save_progress(folder_path)
            
        print(folder_path)
        # load data
        rate, data = wavfile.read(wav_file_path)
        # perform noise reduction
        reduced_noise = nr.reduce_noise(y=data, sr=rate)
        # Extract the audio file name without extension
        audio_file_name = os.path.splitext(os.path.basename(wav_file_path))[0]
        # new file name with folder route
        file_name = os.path.join(folder_path, f'preprocessed_{audio_file_name}.wav')
        wavfile.write(file_name, rate, reduced_noise)
      


In [ ]:
processAudio()